# FlyHomes Data Challenge: Kaggle Zillow Prize
### Brian Henn - September 2018 

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from scipy.stats import randint

In [2]:
# define filename and paths
fname_features_2016 = './data/properties_2016.csv'
fname_features_2017 = './data/properties_2017.csv'
fname_sales_2016 = './data/train_2016_v2.csv'
fname_sales_2017 = './data/train_2017.csv'
fname_sub = './data/sample_submission.csv'

In [3]:
# load 2016 and 2017 sales data
sales_2016 = pd.read_csv(fname_sales_2016, index_col=0, header=0, 
                         parse_dates=[2], infer_datetime_format=True)
sales_2017 = pd.read_csv(fname_sales_2017, index_col=0, header=0, 
                         parse_dates=[2], infer_datetime_format=True)
sales = pd.concat([sales_2016, sales_2017])
sales_set = sales.index.unique().values
print(sales.head(5))

          logerror transactiondate
parcelid                          
11016594    0.0276      2016-01-01
14366692   -0.1684      2016-01-01
12098116   -0.0040      2016-01-01
12643413    0.0218      2016-01-02
14432541   -0.0050      2016-01-02


In [ ]:
# inner join 2016 and 2017 feature data with sales data on parcel id

# first, load 2016 feature data, grabbing only the rows with sales data for memory purposes
iter_csv = pd.read_csv(fname_features_2016, index_col=0, header=0, iterator=True, chunksize=50000)
features_2016 = pd.concat([chunk[chunk.index.isin(sales_set)] for chunk in iter_csv])

# now, join the feature data to the sales data, duplicating properties with multiple sales
features_2016_with_sales = features_2016.merge(sales, left_index=True, right_index=True)

/home/brian/anaconda3/envs/insight/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/brian/anaconda3/envs/insight/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/brian/anaconda3/envs/insight/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (22,49) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
frac_at_least_one_missing = sum(features_2016_with_sales.isnull().sum(1) > 0)/len(features_2016_with_sales)
print('Fraction of rows with at least one missing value: %0.4f.\n' % (frac_at_least_one_missing))

frac_missing = []
for col in features_2016_with_sales.columns.values:
    frac_missing.append((len(features_2016_with_sales[col]) - features_2016_with_sales[col].count())/len(
        features_2016_with_sales[col]))
    
fig = plt.figure()
fig.set_size_inches([8,15])
ax = fig.subplots()
ax.barh(range(len(frac_missing)), frac_missing)
ax.set_xlim([0,1]) 
ax.set_xlabel('Fraction Missing', fontsize=16)
ax.set_ylim([0,len(frac_missing)])
ax.set_xticks(np.arange(0,1.01,0.1))
ax.set_yticks(range(len(frac_missing)))
ax.set_yticklabels(features_2016_with_sales.columns.values,rotation = 0);
ax.grid(True,'major','y')
ax.set_title('Fraction of Missing Data', fontsize=20)

In [ ]:
# reduce features to those with at least 75% of data

features_2016_with_sales = features_2016_with_sales.iloc[:,[frac < 0.25 for frac in frac_missing]]
frac_missing = [frac_missing[i] for i, _ in enumerate(frac_missing) if frac_missing[i] < 0.25]

In [ ]:
print(features_2016_with_sales.shape)
print(features_2016_with_sales.columns)
print(features_2016_with_sales['propertylandusetypeid'].value_counts())

In [ ]:
features_2016_with_sales.iloc[:,:20].sample(15)

In [ ]:
# transform categorical variable about property type into more useful set of dummy variables

# single family homes (most of the dataset)
features_2016_with_sales['single_family'] = [
    1 if val == 261.0 else 0 for val in features_2016_with_sales['propertylandusetypeid']]
# multi-family (duplex/triplex etc.)
features_2016_with_sales['multi_family'] = [
    1 if (val >= 246.0 and val <= 248.0) else 0 for val in features_2016_with_sales['propertylandusetypeid']]
# condos (a lot of these also)
features_2016_with_sales['condominium'] = [
    1 if val == 266.0 else 0 for val in features_2016_with_sales['propertylandusetypeid']]
# planned/cluster/mobile (a few of these)
features_2016_with_sales['planned_community'] = [
    1 if (val == 263.0 or val == 265.0 or val == 269.0) \
    else 0 for val in features_2016_with_sales['propertylandusetypeid']]
# everything else (small number of random unusual residential types and commercial properties)
features_2016_with_sales['other_property'] = [
    1 if val not in [246.0, 247.0, 248.0, 261.0, 263.0, 265.0, 266.0, 
        269.0] else 0 for val in features_2016_with_sales['propertylandusetypeid']]

#features_2016_with_sales[['propertylandusetypeid','single_family','multi_family','condominium']].sample(15)

In [ ]:
# list useful features for inclusion in the model

features_in_model = ['bedroomcnt','bathroomcnt','calculatedfinishedsquarefeet','fullbathcnt',
                     'latitude','longitude','lotsizesquarefeet','single_family','multi_family','condominium',
                    'planned_community','other_property','regionidzip','yearbuilt','taxvaluedollarcnt']

X = features_2016_with_sales[features_in_model]
X.sample(10)

In [ ]:
# impute missing values using sklearn's implementation

# impute using the mean as a simple solution
imputer = Imputer(strategy='mean')
X_imputed = X.copy()
for col in X.columns.values:
    if sum(X[col].isna()) > 0: # only apply this to columns with missing data
        imputer.fit(X[[col]])
        X_imputed[col] = imputer.transform(X[[col]]).ravel()

X_imputed.sample(20)

In [ ]:
# get sales data as training dataset

Y = features_2016_with_sales[['logerror','transactiondate']]
Y.sample(15)

In [ ]:
# examine the correlations among the selected features and with the log error

# https://seaborn.pydata.org/examples/many_pairwise_correlations.html

# set up colormap for correlation plots
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
cmax = 1
cmin = -1

# create a dataframe of just the features we want to look at, plus the target)
plot_columns = features_in_model
plot_columns.append('logerror')
full_df = features_2016_with_sales[plot_columns]
                                                    
# Compute the correlation matrix
corr_properties = full_df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_properties, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_properties, mask=mask, cmap=cmap, vmax=cmax, vmin=cmin, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
# now bar plot just correlation of logerror against features

print(corr_properties.shape)

fig = plt.figure()
fig.set_size_inches([8,6])
ax = fig.subplots()
ax.bar(range(len(full_df.columns.values) - 1), corr_properties.iloc[-1,0:15])
ax.set_ylim([-0.25,0.25])
ax.set_ylabel('Correlation with Log Error', fontsize=16)
ax.set_xticks(range(len(full_df.columns.values) - 1))
ax.set_xticklabels(full_df.columns.values[0:-1],rotation=-90, fontsize=16)
ax.grid(True,'major','y')

In [ ]:
# split sample into training and test sets 
# retain 20% of the properties with sales (34k) as a test dataset, train on 80% (134k)
X_train, X_test, Y_train, Y_test = train_test_split(X_imputed, Y['logerror'], test_size=0.2, shuffle=True)

# Note: for the moment we are not worry about the time dimension of the sales,
# instead just using all of the sales from 2016 and 2017 as both training and validation data

In [ ]:
# let's train the gradient boosted tree model's hyperparameters using sklearn's RandomSearchCV capabilities

# define the estimator as GBT
gbt = GradientBoostingRegressor(loss='lad')

# define the hyperparameter we want to search across in random sampling (as in section 2)
param_dist = {'n_estimators' : randint(2, 50), # number of boosting iterations 
              "max_depth": randint(3, 10), # allow for potentially many (100 splits) in the trees
              "max_features": [None, 'sqrt'], # allow either any number of features in bagging, or just the sqrt
              "min_samples_split": randint(2, 11), # controls on splitting and leaves
              "min_samples_leaf": randint(1, 11),
              "criterion": ['friedman_mse', 'rmse', 'mae']} # vary the scoring method
# random sampling strategy suggested by scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py

# randomly search parameters using my laptop's cores
n_iter_random = 20 # 5 iterations of parameters for each fold 
cv_random = 5 # 5-fold cross validation
gbt_hypertuning = RandomizedSearchCV(estimator=gbt, param_distributions=param_dist, n_iter=n_iter_random,
                               cv=cv_random, random_state=91214, n_jobs=-1, verbose=4, return_train_score=True)

# Fit the random search model for hyperparameter tuning
gbt_hypertuning.fit(X_train, Y_train)

In [ ]:
results_df = pd.DataFrame(gbt_hypertuning.cv_results_)
results_df.sort_values('mean_test_score', ascending=False, inplace=True)
results_df.head(5)

In [ ]:
# train model using best hyperparameters and whole training dataset

# use the best hyperparameters
best = gbt_hypertuning.best_params_
print(best)
gbt_train = GradientBoostingRegressor(**best)

# train the model
gbt_train.fit(X_train,Y_train)

In [ ]:
# evaluate best model on test dataset

# produce predictions 
Y_pred = gbt_train.predict(X_test)

# compute MAE on predictions
MAE = mean_absolute_error(Y_pred, Y_test)
print(MAE)

gbt_train.score(X_test, Y_test, sample_weight=None)

In [ ]:
fig, ax = plt.subplots()
ax.plot(Y_test, Y_pred, 'b +')